In [ ]:
import requests
import json
import os

# --- Configuration (Replace with your actual values) ---
# IMPORTANT: Never hardcode sensitive information like access tokens in production code.
# Use environment variables or a secure configuration management system.
THREADS_USER_ID = "YOUR_THREADS_USER_ID"  # The numerical ID of the Threads account
ACCESS_TOKEN = "YOUR_LONG_LIVED_ACCESS_TOKEN" # Your long-lived access token

# Meta Graph API base URL for Threads
GRAPH_API_BASE_URL = "https://graph.threads.net/v1.0"

def create_threads_post(user_id, access_token, text_content=None, media_url=None, is_carousel=False, children_media_urls=None):
    """
    Creates a new post on Threads.

    Args:
        user_id (str): The Threads User ID of the account to post to.
        access_token (str): The long-lived access token for the user.
        text_content (str, optional): The text content of the post. Max 500 characters.
        media_url (str, optional): URL of a single image or video for the post.
                                   Media must be publicly accessible.
        is_carousel (bool, optional): Set to True if creating a carousel post.
        children_media_urls (list, optional): List of URLs for images/videos in a carousel post.
                                              Requires is_carousel=True. Max 20 children.

    Returns:
        dict: The API response as a dictionary, or None if an error occurred.
    """
    if not user_id or not access_token:
        print("Error: Threads User ID and Access Token are required.")
        return None

    headers = {
        "Content-Type": "application/json"
    }

    # Step 1: Create a media container (required for all media posts, and for text posts with links)
    # For a simple text post without media or link, this step is skipped.
    media_container_id = None
    if media_url or is_carousel:
        container_endpoint = f"{GRAPH_API_BASE_URL}/{user_id}/threads_publishing"
        container_params = {
            "access_token": access_token
        }

        if is_carousel and children_media_urls:
            # Create individual media containers for each child in the carousel
            child_container_ids = []
            for url in children_media_urls:
                child_params = {
                    "media_url": url,
                    "is_carousel_item": True,
                    "access_token": access_token
                }
                try:
                    child_response = requests.post(container_endpoint, params=child_params)
                    child_response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
                    child_data = child_response.json()
                    if 'id' in child_data:
                        child_container_ids.append(child_data['id'])
                    else:
                        print(f"Error creating child media container for {url}: {child_data}")
                        return None
                except requests.exceptions.RequestException as e:
                    print(f"Request error creating child media container for {url}: {e}")
                    return None

            if not child_container_ids:
                print("Error: No child media containers were successfully created for the carousel.")
                return None

            # Create the carousel container using child media IDs
            carousel_params = {
                "children": ",".join(child_container_ids), # Comma-separated list of child container IDs
                "media_type": "CAROUSEL",
                "access_token": access_token
            }
            try:
                carousel_response = requests.post(container_endpoint, params=carousel_params)
                carousel_response.raise_for_status()
                carousel_data = carousel_response.json()
                if 'id' in carousel_data:
                    media_container_id = carousel_data['id']
                    print(f"Carousel container created with ID: {media_container_id}")
                else:
                    print(f"Error creating carousel container: {carousel_data}")
                    return None
            except requests.exceptions.RequestException as e:
                print(f"Request error creating carousel container: {e}")
                return None

        elif media_url:
            # Single image/video post
            container_params["media_url"] = media_url
            # The API infers media_type from the URL content
            try:
                response = requests.post(container_endpoint, params=container_params)
                response.raise_for_status()
                data = response.json()
                if 'id' in data:
                    media_container_id = data['id']
                    print(f"Media container created with ID: {media_container_id}")
                else:
                    print(f"Error creating media container: {data}")
                    return None
            except requests.exceptions.RequestException as e:
                print(f"Request error creating media container: {e}")
                return None

    # Step 2: Publish the post
    publish_endpoint = f"{GRAPH_API_BASE_URL}/{user_id}/threads_publish"
    publish_params = {
        "access_token": access_token
    }

    if text_content:
        publish_params["text"] = text_content
    if media_container_id:
        publish_params["media_container_id"] = media_container_id

    # Handle text-only posts or posts with media
    if not text_content and not media_container_id:
        print("Error: Either 'text_content' or 'media_url' (or 'children_media_urls' for carousel) must be provided.")
        return None

    try:
        response = requests.post(publish_endpoint, params=publish_params)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        result = response.json()
        print("Post creation response:")
        print(json.dumps(result, indent=4))
        return result
    except requests.exceptions.RequestException as e:
        print(f"Request error during post publishing: {e}")
        if hasattr(e, 'response') and e.response is not None:
            print(f"Response content: {e.response.text}")
        return None

# --- Example Usage ---
if __name__ == "__main__":
    # Load credentials from environment variables for security
    # It's highly recommended to set these in your system's environment
    # For example:
    # export THREADS_USER_ID="1234567890"
    # export THREADS_ACCESS_TOKEN="EAAG..."
    
    # Placeholder for actual credentials. DO NOT USE THESE IN PRODUCTION.
    # You MUST replace these with your actual Threads User ID and Access Token.
    # For local testing, you can uncomment and set them directly, but be cautious.
    # THREADS_USER_ID = "YOUR_ACTUAL_THREADS_USER_ID"
    # ACCESS_TOKEN = "YOUR_ACTUAL_LONG_LIVED_ACCESS_TOKEN"

    # Fetch from environment variables if set, otherwise use placeholders
    user_id = os.getenv("THREADS_USER_ID", THREADS_USER_ID)
    access_token = os.getenv("THREADS_ACCESS_TOKEN", ACCESS_TOKEN)

    if user_id == "YOUR_THREADS_USER_ID" or access_token == "YOUR_LONG_LIVED_ACCESS_TOKEN":
        print("Please replace 'YOUR_THREADS_USER_ID' and 'YOUR_LONG_LIVED_ACCESS_TOKEN' with your actual credentials.")
        print("Alternatively, set them as environment variables: THREADS_USER_ID and THREADS_ACCESS_TOKEN.")
    else:
        print(f"Attempting to post as Threads User ID: {user_id[:5]}...") # Show only first 5 chars for privacy

        # --- Example 1: Create a simple text post ---
        print("\n--- Creating a text post ---")
        post_text = "Hello Threads from Python! This is a test post using the official API."
        create_threads_post(user_id, access_token, text_content=post_text)

        # --- Example 2: Create a post with a single image ---
        # NOTE: The image URL MUST be publicly accessible.
        # Replace with a real, publicly accessible image URL for testing.
        # For example: https://placehold.co/600x400/FF0000/FFFFFF?text=Python+Test
        print("\n--- Creating an image post ---")
        image_post_text = "Check out this image posted from Python!"
        image_url = "https://placehold.co/600x400/FF0000/FFFFFF?text=Python+Test"
        # create_threads_post(user_id, access_token, text_content=image_post_text, media_url=image_url)

        # --- Example 3: Create a carousel post with multiple images ---
        # NOTE: All image URLs MUST be publicly accessible.
        print("\n--- Creating a carousel post (requires multiple public image URLs) ---")
        carousel_post_text = "This is a carousel post from Python! Image 1, Image 2."
        carousel_image_urls = [
            "https://placehold.co/600x400/00FF00/FFFFFF?text=Carousel+1",
            "https://placehold.co/600x400/0000FF/FFFFFF?text=Carousel+2"
        ]
        # create_threads_post(user_id, access_token, text_content=carousel_post_text,
        #                     is_carousel=True, children_media_urls=carousel_image_urls)

